In [1]:
!pip cache purge
!pip uninstall -y torch torchtext fastai timm torchvision torchaudio torchdata
!pip install torch==2.3.0 torchtext==0.18.0

Files removed: 0
Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: fastai 2.7.18
Uninstalling fastai-2.7.18:
  Successfully uninstalled fastai-2.7.18
Found existing installation: timm 1.0.11
Uninstalling timm-1.0.11:
  Successfully uninstalled timm-1.0.11
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://www.kaggle.com/code/gmhost/gru-capsule

Sentiment analysis based on improved pre-trained word embeddings
https://www.sciencedirect.com/science/article/abs/pii/S095741741830558X

In [3]:
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict,Counter
import re
import numpy as np
import spacy
import pickle
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler
from sklearn.feature_extraction.text import CountVectorizer
from torchtext.vocab import GloVe
from sklearn.metrics import accuracy_score, f1_score

/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


### **Custom Word Positivity Score:**

In [ ]:
import spacy
import re
from collections import defaultdict
import pandas as pd

nlp = spacy.load("en_core_web_sm")
label_map = {-1: 'negative', 0: 'neutral', 1: 'positive'}
stopwords = {"a", "the", "and", "is", "in", "to", "for", "of", "on", "it", "this", "that", "with", "as", "at", "by", "an"}
min_frequency = 10
relevant_pos_tags = {"ADJ", "ADV", "VERB", "NOUN"}

word_counts = defaultdict(lambda: {'positive': 0, 'neutral': 0, 'negative': 0})
word_frequencies = defaultdict(int)

df = pd.read_csv("/content/train.csv")

sentences = df['sentence'].str.lower().tolist()
labels = df['gold_label'].tolist()

for doc, label in zip(nlp.pipe(sentences, batch_size=32), labels):
    sentiment = label_map.get(label, 'neutral')
    for token in doc:
        if token.is_alpha and token.pos_ in relevant_pos_tags and token.text not in stopwords:
            word_counts[token.text][sentiment] += 1
            word_frequencies[token.text] += 1

word_scores = {}
for word, counts in word_counts.items():
    if word_frequencies[word] >= min_frequency:
        total_count = counts['positive'] + counts['neutral'] + counts['negative']
        if total_count > 0:
            score = (counts['positive'] - counts['negative']) / total_count
            word_scores[word] = score

word_scores_df = pd.DataFrame(list(word_scores.items()), columns=['word', 'positivity_score'])
word_scores_df = word_scores_df.sort_values(by='positivity_score', ascending=False)
output_path = 'word_positivity_scores.csv'
word_scores_df.to_csv(output_path, index=False)
print(f"Word positivity scores saved to {output_path}")


###**GRU+ Glove + Custom Positivity score + POS :**

In [4]:
nlp = spacy.load("en_core_web_sm")
glove = GloVe(name='6B', dim=100)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

.vector_cache/glove.6B.zip: 862MB [02:38, 5.43MB/s]                           
100%|█████████▉| 399999/400000 [00:20<00:00, 19080.04it/s]


Using device: cuda


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

positivity_scores_df = pd.read_csv("/content/drive/MyDrive/NLP/Sentiment_Analysis_Competition/word_positivity_scores.csv")
positivity_scores = dict(zip(positivity_scores_df['word'], positivity_scores_df['positivity_score']))

pos_tags = ["NOUN", "VERB", "ADJ", "ADV", "PRON", "DET", "ADP", "NUM", "CONJ", "PART", "INTJ", "AUX"]
pos_embedding_layer = nn.Embedding(len(pos_tags), 30)
pos_to_idx = {tag: idx for idx, tag in enumerate(pos_tags)}

dep_tags = list(nlp.get_pipe("parser").labels)
dep_embedding_layer = nn.Embedding(len(dep_tags), 30)
dep_to_idx = {dep: idx for idx, dep in enumerate(dep_tags)}

negation_words = {
    "not", "no", "never", "n't", "cannot", "can't", "won't",
    "didn't", "doesn't", "isn't", "aren't", "wasn't", "weren't",
    "shouldn't", "wouldn't", "couldn't", "mustn't", "mightn't",
    "needn't", "without", "nor", "neither", "nowhere", "none", "nothing", "nobody", "hardly", "scarcely", "barely"
}

def get_dependency_contextual_embedding(tokens, index):
    token = tokens[index]
    dependent_tokens = [child for child in token.children]
    context_embeddings = []
    for dep_token in dependent_tokens:
        lower_word = dep_token.text.lower()
        embedding = glove[lower_word] if lower_word in glove.stoi else torch.zeros(100)
        context_embeddings.append(embedding)
    if context_embeddings:
        return torch.mean(torch.stack(context_embeddings), dim=0)
    return torch.zeros(100)


def get_token_embedding(token, tokens):
    original_word = token.text
    lower_word = original_word.lower()
    if lower_word in glove.stoi:
        embedding = glove[lower_word]
        if torch.sum(embedding) == 0.0:
            embedding = torch.randn(100)
    else:
        embedding = torch.randn(100)

    # POS Embedding
    pos_index = pos_to_idx.get(token.pos_, len(pos_tags) - 1)
    pos_embedding = pos_embedding_layer(torch.tensor(pos_index)).detach()

    # Positivity Embedding
    positivity_score = 5 * positivity_scores.get(lower_word, 0.0)
    positivity_embedding = torch.tensor([positivity_score], dtype=torch.float32)

    # Dependency Embedding
    dep_index = dep_to_idx.get(token.dep_, len(dep_tags) - 1)
    dep_embedding = dep_embedding_layer(torch.tensor(dep_index)).detach()

    # Negation Feature
    negation = 1.0 if any(ancestor.text.lower() in negation_words for ancestor in token.ancestors) else 0.0
    negation_embedding = torch.tensor([negation], dtype=torch.float32)

    # Contextual Embedding
    contextual_embedding = get_dependency_contextual_embedding(tokens, token.i) if len(tokens) > 1 else torch.zeros(100)

    # Punctuation Feature
    punctuation = 1.0 if any(punct in token.sent.text for punct in {"!", "?"}) else 0.0
    punctuation_embedding = torch.tensor([punctuation], dtype=torch.float32)

    # Capitalization Feature
    capitalization = 1.0 if original_word.isupper() else 0.0
    capitalization_embedding = torch.tensor([capitalization], dtype=torch.float32)

    return torch.cat((
        embedding,
        contextual_embedding,
        pos_embedding,
        dep_embedding,
        positivity_embedding,
        negation_embedding,
        punctuation_embedding,
        capitalization_embedding
    ))

df = pd.read_csv("/content/drive/MyDrive/NLP/Sentiment_Analysis_Competition/Data/train.csv")
df['gold_label'] = df['gold_label'].map({-1: 0, 0: 1, 1: 2})


MAX_TOKENS = 64
from tqdm import tqdm

precomputed_data = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing Sentences"):
    text = row['sentence']
    label = row['gold_label']
    doc = nlp(text)
    limited_tokens = list(doc)[:MAX_TOKENS]
    embeddings = [get_token_embedding(token, doc) for token in limited_tokens]
    precomputed_data.append((torch.stack(embeddings), label))


Processing Sentences: 100%|██████████| 92228/92228 [18:55<00:00, 81.22it/s]


**TextCNN:**

In [7]:
labels = [item[1] for item in precomputed_data]
train_data, val_data = train_test_split(precomputed_data, test_size=0.1, stratify=labels, random_state=42)
import torch.nn.functional as F

class SentimentDatasetWrapper(Dataset):
    def __init__(self, embeddings, labels=None):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.embeddings[idx], self.labels[idx]
        return self.embeddings[idx]

class TextCNN(nn.Module):
    def __init__(self, input_features, output_classes, filter_sizes=[3, 4, 5], num_filters=100, dropout_probability=0.5):
        super(TextCNN, self).__init__()
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (filter_size, input_features))
            for filter_size in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * num_filters, output_classes)
        self.dropout = nn.Dropout(dropout_probability)

    def forward(self, input_tensor):
        input_tensor = input_tensor.unsqueeze(1)
        conv_outs = [
            F.relu(conv(input_tensor)).squeeze(3)
            for conv in self.convs
        ]
        pooled_outs = [
            F.max_pool1d(conv_out, conv_out.size(2)).squeeze(2)
            for conv_out in conv_outs
        ]
        concatenated = torch.cat(pooled_outs, dim=1)
        dropped_out = self.dropout(concatenated)
        return self.fc(dropped_out)


# Training and Validation
def train_and_validate(train_data, val_data, model, epochs=40, batch_size=64, patience_threshold = 5):
    training_dataset = SentimentDatasetWrapper([x[0] for x in train_data], [x[1] for x in train_data])
    validation_dataset = SentimentDatasetWrapper([x[0] for x in val_data], [x[1] for x in val_data])

    training_data_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda batch: zip(*batch))
    validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda batch: zip(*batch))

    loss_function = nn.CrossEntropyLoss()
    optimizer_instance = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    learning_rate_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_instance, mode='min', patience=5, verbose=True)

    highest_validation_f1 = 0
    early_stopping_counter = 0

    for epoch in range(epochs):
        model.train()
        training_losses = []
        ground_truth_train = []
        predicted_train = []

        for batch_inputs, batch_labels in training_data_loader:
            padded_inputs = torch.nn.utils.rnn.pad_sequence(batch_inputs, batch_first=True).to(device)
            tensor_labels = torch.tensor(batch_labels).to(device)

            optimizer_instance.zero_grad()
            model_outputs = model(padded_inputs)
            loss_value = loss_function(model_outputs, tensor_labels)
            loss_value.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer_instance.step()

            training_losses.append(loss_value.item())
            _, predictions = torch.max(model_outputs, 1)
            ground_truth_train.extend(tensor_labels.cpu().numpy())
            predicted_train.extend(predictions.cpu().numpy())

        training_accuracy = accuracy_score(ground_truth_train, predicted_train) * 100
        training_f1_score = f1_score(ground_truth_train, predicted_train, average='weighted') * 100
        average_training_loss = np.mean(training_losses)

        model.eval()
        validation_losses = []
        ground_truth_val = []
        predicted_val = []

        with torch.no_grad():
            for batch_inputs, batch_labels in validation_data_loader:
                padded_inputs = torch.nn.utils.rnn.pad_sequence(batch_inputs, batch_first=True).to(device)
                tensor_labels = torch.tensor(batch_labels).to(device)

                model_outputs = model(padded_inputs)
                loss_value = loss_function(model_outputs, tensor_labels)
                validation_losses.append(loss_value.item())
                _, predictions = torch.max(model_outputs, 1)
                ground_truth_val.extend(tensor_labels.cpu().numpy())
                predicted_val.extend(predictions.cpu().numpy())

        average_validation_loss = np.mean(validation_losses)
        validation_accuracy = accuracy_score(ground_truth_val, predicted_val) * 100
        validation_f1_score = f1_score(ground_truth_val, predicted_val, average='weighted') * 100
        learning_rate_scheduler.step(average_validation_loss)

        print(f"Epoch {epoch + 1} - "
              f"Train Loss: {average_training_loss:.4f}, Train Accuracy: {training_accuracy:.2f}%, Train F1: {training_f1_score:.2f}%, "
              f"Val Loss: {average_validation_loss:.4f}, Val Accuracy: {validation_accuracy:.2f}%, Val F1: {validation_f1_score:.2f}%")

        if validation_f1_score > highest_validation_f1:
            highest_validation_f1 = validation_f1_score
            early_stopping_counter = 0
            torch.save(model.state_dict(), 'best_model_state.pt')
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience_threshold:
                print("Early stopping triggered.")
                break

    model.load_state_dict(torch.load('best_model_state.pt'))
    print(f"Best Validation F1 Score: {highest_validation_f1:.2f}%")

model = TextCNN(
    input_features=264,
    output_classes=3,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    dropout_probability=0.5
).to(device)

train_and_validate(train_data, val_data, model, epochs=40, batch_size=32, patience_threshold=5)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1 - Train Loss: 0.8100, Train Accuracy: 65.44%, Train F1: 64.08%, Val Loss: 0.7435, Val Accuracy: 68.38%, Val F1: 66.83%
Epoch 2 - Train Loss: 0.7598, Train Accuracy: 68.24%, Train F1: 67.11%, Val Loss: 0.7289, Val Accuracy: 69.78%, Val F1: 68.74%
Epoch 3 - Train Loss: 0.7427, Train Accuracy: 68.96%, Train F1: 67.90%, Val Loss: 0.7282, Val Accuracy: 69.23%, Val F1: 67.57%
Epoch 4 - Train Loss: 0.7254, Train Accuracy: 69.90%, Train F1: 68.89%, Val Loss: 0.7199, Val Accuracy: 69.72%, Val F1: 68.75%
Epoch 5 - Train Loss: 0.7140, Train Accuracy: 70.41%, Train F1: 69.45%, Val Loss: 0.7262, Val Accuracy: 69.36%, Val F1: 67.76%
Epoch 6 - Train Loss: 0.7006, Train Accuracy: 70.96%, Train F1: 70.05%, Val Loss: 0.7192, Val Accuracy: 69.84%, Val F1: 68.24%
Epoch 7 - Train Loss: 0.6906, Train Accuracy: 71.42%, Train F1: 70.58%, Val Loss: 0.7193, Val Accuracy: 69.64%, Val F1: 68.27%
Epoch 8 - Train Loss: 0.6777, Train Accuracy: 72.01%, Train F1: 71.22%, Val Loss: 0.7147, Val Accuracy: 69.94%,

In [8]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from tqdm import tqdm

dev_df = pd.read_csv("/content/drive/MyDrive/NLP/Sentiment_Analysis_Competition/Data/dev.csv")
precomputed_dev_data = []

for _, row in tqdm(dev_df.iterrows(), total=len(dev_df)):
    text = row['sentence']
    doc = nlp(text)
    limited_tokens = list(doc)[:MAX_TOKENS]
    embeddings = [get_token_embedding(token, doc) for token in limited_tokens]
    precomputed_dev_data.append(torch.stack(embeddings))

dev_dataset = SentimentDatasetWrapper(precomputed_dev_data)
dev_data_loader = DataLoader(dev_dataset, batch_size=64, shuffle=False, collate_fn=lambda batch: batch)
model.load_state_dict(torch.load('best_model_state.pt'))
model.eval()
predictions = []

with torch.no_grad():
    for batch_inputs in tqdm(dev_data_loader):
        padded_inputs = torch.nn.utils.rnn.pad_sequence(batch_inputs, batch_first=True).to(device)
        outputs = model(padded_inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())
adjusted_predictions = [pred - 1 for pred in predictions]
with open('answer.txt', 'w') as f:
    for pred in adjusted_predictions:
        f.write(f"{pred}\n")

print("Predictions saved to answer.txt")


100%|██████████| 76/76 [00:00<00:00, 198.34it/s]

Predictions saved to answer.txt
